In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint
from yaml import safe_load
import torch.nn as nn

exphash = '6c82028bc'

exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

ds = InitDataset(h)

#h['hyperparameters']['manual_attention'] = True
model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[16:24:28] loading configuration file bert_distil_uncased/config.json


[16:24:28] loading weights file bert_distil_uncased/pytorch_model.bin


[16:24:30] Loaded model: <All keys matched successfully>


In [2]:
h['dataset_split'][1] = .01

In [3]:
val_loader = DataLoader(ds[1], batch_size=64, shuffle=False)


In [4]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [5]:
xent = nn.CrossEntropyLoss()
def xentropy_loss_fn(output, labels):
  return xent(output.view(-1, output.size(-1)), labels.view(-1))

def top1_acc_fn(output, labels):
  top = output.argmax(-1)
  right = top==labels
  return right.float()

def Validate(val_loader, model):
  cum_acc = 0
  cum_xent_loss = 0

  rights_ = []
  inputs_ =[]
  labels_ =[]
  maxes_ =[]
  for i, data in enumerate(val_loader):
    inputs = data['input_ids']
    labels = data['label_ids']
    inputs_ += Torch2Py(inputs)
    labels_ += Torch2Py(labels)
    
    with torch.no_grad():
      model.eval()
      outputs = model(inputs)
      maxes_ += Torch2Py(outputs.argmax(-1))
      xent_loss = xentropy_loss_fn(outputs, labels)
      cum_xent_loss += xent_loss      

      top1_acc = top1_acc_fn(outputs, labels)
      rights_ += Torch2Py(top1_acc)
      cum_acc += top1_acc.mean()
    if len(inputs_)>100:
      break

  steps = i+1
  print(cum_acc/steps, cum_xent_loss.detach()/steps)
  return rights_, inputs_, labels_, maxes_,



In [6]:
right, inputs, labels, maxes = Validate(val_loader, model)

tensor(0.0125, device='cuda:0') tensor(9.7464, device='cuda:0')


In [7]:
len(right), len(inputs), len(labels), len(maxes),

(128, 128, 128, 128)

In [8]:
model.embedded_chars[0].shape

torch.Size([96, 768])

In [9]:
char_to_idx_map = torch.load('../data/char_to_idx_map.pt')

In [10]:
idx_to_char_map= {v:k for k,v in char_to_idx_map.items()}

In [11]:
idx_to_tok_map = {}
with open('../data/bert-base-uncased-vocab.txt') as f:
  for i, l in enumerate(f.readlines()):
    idx_to_tok_map[i] = l.strip()

In [12]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [13]:
def GetToks(encoded):
  s = ""
  for c in encoded:
    s+= " " + idx_to_tok_map[c]
  return s

In [105]:
inps = ds[0][0:3]['input_ids']

In [14]:
with torch.no_grad():
  a = ((model.embedded_chars[0, 0:8, :].unsqueeze(1)-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [15]:
with torch.no_grad():
  a = (model.embedded_chars[:3, 0:20, :].unsqueeze(1)@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)

In [16]:
print(GetChars(inputs[0]))

[cls] so off the wall and hilarious. you would never expect that guys moving around action figures to do things would be hilarious, but it really is. this is honestly one of the best shows adult swim has put out since they began to air shows like space ghost coast to coast and family guy. creators, including seth green________________________________________________________________


In [17]:
print(GetToks(maxes[0]))

 [CLS] dax / so off the wall and hilarious ; you would never expect that guys moving around action figures ##ta do high would ##b ##hul 2 - ##i ##dda ##isa ##i [CLS] ##ys ##hosh ##ld ##oth ##oth ##ayo ##bes show shah ##sz pas put put ##oud pei ##kle ##dna ##oe ##sio auschwitz 3 horn coast go ##jo ##jas bat tao 3 2 2 2 2 57 greens green ##pw ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy ##wy


In [225]:
embeddings = ds.data['embeddings']

In [226]:
embeddings.norm(2,-1).mean()

tensor(1.6638, device='cuda:0')

In [252]:
embedding_noise = .25

In [253]:
r = torch.normal(mean=embedding_noise, std=1, size=embeddings.size()).cuda()
norm_factor = torch.norm(r, 2, -1).mean()/embedding_noise
normalized_noise = r/norm_factor
new = embeddings + normalized_noise

In [254]:
((new-embeddings)**2).mean()

tensor(8.1433e-05, device='cuda:0')

In [255]:
torch.sqrt((new-embeddings)**2).mean()

tensor(0.0072, device='cuda:0')

In [217]:
n = new.norm(2,-1)

In [31]:
def MakeHist(h, name=None):
  y = h[0]
  x= h[1]
  return {
    'name': name,
    'type': 'bar',                                                                                                                   'x': x,
          'marker': dict(opacity=.7),
   'y': y
  }

In [30]:
dat = MakeHist(np.histogram(n, bins=30), name='toks')

fig = fig = go.Figure(data=[dat], layout={'barmode':'overlay'})
plotly.offline.plot(fig, filename='/home/tucker/Downloads/imdb_size_distci.html')


NameError: name 'n' is not defined

In [22]:
attn = torch.nn.Sigmoid()(model.char_embedder.char_block_attn)

In [20]:
sb = model.char_embedder.switchboard[0]

In [24]:
hm = go.Heatmap(z=sb.detach().cpu().numpy())
fig = go.Figure(data=[hm])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/strategy_heatmap.html')

'/home/tucker/Downloads/strategy_heatmap.html'

In [25]:
attn.shape

torch.Size([64, 384])

In [33]:
attn_acts = attn.reshape(-1).detach().cpu().numpy()

In [34]:
dat = MakeHist(np.histogram(attn_acts, bins=30), name='toks')
fig = fig = go.Figure(data=[dat], layout={'barmode':'overlay'})
plotly.offline.plot(fig, filename='/home/tucker/Downloads/act_dist.html')


'/home/tucker/Downloads/act_dist.html'

In [21]:
sb.shape

torch.Size([96, 384])

In [50]:
chrs = []
for enc in inputs[-1:]:
  chrs.append(GetChars(enc))


In [51]:
chrs[0]

"the writer's intent. there's a lot of historical, literary, and biblical allusions which add to the complexity of the movie, and ultimately i think it adds depth and helps us to understand the transformations of the characters. the movie is all the more amazing when you consider it is based on a true story. i would recommend this_____________________________________________________"

In [23]:
hm = go.Heatmap(z=attn.detach().cpu().numpy())
fig = go.Figure(data=[hm])
plotly.offline.plot(fig, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [40]:
count = 5

In [41]:
import plotly.figure_factory as ff
hm = ff.create_annotated_heatmap(
  z=attn[:count].detach().cpu().numpy(), annotation_text=chrs[:count],
  text=attn[:count].detach().cpu().numpy(), hoverinfo='text')

plotly.offline.plot(hm, filename='/home/tucker/Downloads/strategy_heatmap.html')


'/home/tucker/Downloads/strategy_heatmap.html'

In [35]:
from experiment import IdentityConv

In [46]:
idc = IdentityConv(3)

In [47]:
sequence = torch.tensor([[1,2,3,4,5,6,7]])

In [48]:
emb = torch.nn.Embedding(10, 4)

In [49]:
embedded = emb(sequence)

In [50]:
embedded.shape

torch.Size([1, 7, 4])

In [52]:
embedded

tensor([[[ 0.2802,  0.1562, -1.0467, -0.2609],
         [-1.4873, -0.2946,  0.3285,  0.0856],
         [ 0.2202, -1.6364, -0.4096, -0.6284],
         [ 0.3340, -0.0501,  2.0443,  0.1030],
         [ 1.4272, -2.0116,  0.3527, -1.3549],
         [ 0.7472, -1.8266, -0.9577, -1.8917],
         [-0.2680, -0.1245, -1.2108,  0.2656]]], grad_fn=<EmbeddingBackward>)

In [53]:
uf = idc(embedded.permute(0,2,1))

In [54]:
uf, uf.shape

(tensor([[[ 0.2802, -1.4873,  0.2202,  0.3340,  1.4272],
          [-1.4873,  0.2202,  0.3340,  1.4272,  0.7472],
          [ 0.2202,  0.3340,  1.4272,  0.7472, -0.2680],
          [ 0.1562, -0.2946, -1.6364, -0.0501, -2.0116],
          [-0.2946, -1.6364, -0.0501, -2.0116, -1.8266],
          [-1.6364, -0.0501, -2.0116, -1.8266, -0.1245],
          [-1.0467,  0.3285, -0.4096,  2.0443,  0.3527],
          [ 0.3285, -0.4096,  2.0443,  0.3527, -0.9577],
          [-0.4096,  2.0443,  0.3527, -0.9577, -1.2108],
          [-0.2609,  0.0856, -0.6284,  0.1030, -1.3549],
          [ 0.0856, -0.6284,  0.1030, -1.3549, -1.8917],
          [-0.6284,  0.1030, -1.3549, -1.8917,  0.2656]]],
        grad_fn=<PermuteBackward>),
 torch.Size([1, 12, 5]))

In [72]:
sb = torch.tensor([[[0,1,0,0,0]]]).float()

In [73]:
sb.shape

torch.Size([1, 1, 5])

In [75]:
dense = sb@uf.permute(0,2,1)

In [77]:
dense.shape

torch.Size([1, 1, 12])